In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0-beta1'

# Tensorflow 安装与配置

## Tensorflow初体验

In [3]:
A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6], [7, 8]])
C = tf.matmul(A, B)

print(C)

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


# Tensorflow 基础

In [4]:
# 定义一个随机数（标量）
random_float = tf.random.uniform(shape=())
random_float

<tf.Tensor: id=10, shape=(), dtype=float32, numpy=0.5348927>

In [5]:
# 定义一个有5个元素的零向量
zero_vector = tf.zeros(shape=(5))
zero_vector

<tf.Tensor: id=14, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>

In [6]:
# 定义两个2×2的常量矩阵
A = tf.constant([[1., 2.], [3., 4.]])
B = tf.constant([[5., 6.], [7., 8.]])

In [7]:
print(A.shape)      # 输出(2, 2)，即矩阵的长和宽均为2
print(A.dtype)      # 输出<dtype: 'float32'>
print(A.numpy())    # 输出[[1. 2.]
            #    [3. 4.]]

(2, 2)
<dtype: 'float32'>
[[1. 2.]
 [3. 4.]]


## 加减乘除

In [11]:
# 计算矩阵A和B的和
C = tf.add(A, B)   
# 计算矩阵A和B的乘积 
D = tf.matmul(A, B) 
C

<tf.Tensor: id=25, shape=(2, 2), dtype=float32, numpy=
array([[ 6.,  8.],
       [10., 12.]], dtype=float32)>

In [12]:
D

<tf.Tensor: id=26, shape=(2, 2), dtype=float32, numpy=
array([[19., 22.],
       [43., 50.]], dtype=float32)>

In [8]:
tf.subtract(A,B)

<tf.Tensor: id=19, shape=(2, 2), dtype=float32, numpy=
array([[-4., -4.],
       [-4., -4.]], dtype=float32)>

In [9]:
tf.divide(A,B)

<tf.Tensor: id=21, shape=(2, 2), dtype=float32, numpy=
array([[0.2       , 0.33333334],
       [0.42857143, 0.5       ]], dtype=float32)>

## 函数$y=x^2$梯度求导当x=3时的导数

In [13]:
#x=3
x = tf.Variable(initial_value=3.)

with tf.GradientTape() as tape:     # 在 tf.GradientTape() 的上下文内，所有计算步骤都会被记录以用于求导
    y = tf.square(x)
y_grad = tape.gradient(y, x)        # 计算y关于x的导数
print([y, y_grad])

[<tf.Tensor: id=38, shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: id=42, shape=(), dtype=float32, numpy=6.0>]


## 函数$L(w,b)=||Xw+b-y||^2$求导

In [9]:
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])        # 计算L(w, b)关于w, b的偏导数
print([L.numpy(), w_grad.numpy(), b_grad.numpy()])

[125.0, array([[ 70.],
       [100.]], dtype=float32), 30.0]


## 线性回归

In [11]:
import numpy as np

X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

构建$y=ax+b$的函数，损失函数是$L(a,b) = (\hat y - y)^2$

In [12]:
a, b = 0, 0
num_epoch = 10000
learning_rate = 1e-3
for e in range(num_epoch):
    # 手动计算损失函数关于自变量（模型参数）的梯度
    y_pred = a * X + b
    grad_a, grad_b = 2*np.dot((y_pred - y),X), 2*np.sum(y_pred - y)

    # 更新参数
    a, b = a - learning_rate * grad_a, b - learning_rate * grad_b

print(a, b)

0.9817775010317537 0.054568006679415956


In [13]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
for e in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred - y))
    # TensorFlow自动计算损失函数关于自变量（模型参数）的梯度
    grads = tape.gradient(loss, variables)
    # TensorFlow自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

print(a, b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.9817748> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0545703>


# Tensorflow模型构建与训练

In [14]:

X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

#y=ax+b
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )

    def call(self, input):
        output = self.dense(input)
        return output


# 以下代码结构与前节类似
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)      # 调用模型 y_pred = model(X) 而不是显式写出 y_pred = a * X + b
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量[a,b]
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear/dense/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.40784496],
       [1.191065  ],
       [1.9742855 ]], dtype=float32)>, <tf.Variable 'linear/dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.78322077], dtype=float32)>]


# Tensorfow 常用模块

In [15]:
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.save("./save/model.ckpt")

'./save/model.ckpt-1'

## 加载模型

In [16]:
model = Linear()
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint("./save"))

In [17]:
X = tf.constant([[1.0, 8.0, 4.0], [4.0, 5.0, 6.0]])
test_pred = model.predict(X)
test_pred

array([[18.616726],
       [20.215637]], dtype=float32)

## TensorBoard可视化

In [18]:
summary_writer = tf.summary.create_file_writer('./tensorboard')
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)      # 调用模型 y_pred = model(X) 而不是显式写出 y_pred = a * X + b
        loss = tf.reduce_mean(tf.square(y_pred - y))
        with summary_writer.as_default():
            tf.summary.scalar("loss", loss, step=i)

    grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear_2/dense_2/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[ 1.9571737 ],
       [-0.03056878],
       [ 2.0183392 ]], dtype=float32)>, <tf.Variable 'linear_2/dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.21406719], dtype=float32)>]


In [19]:
!tensorboard --logdir=./tensorboard 

TensorBoard 1.14.0a20190603 at http://hcq:6006/ (Press CTRL+C to quit)
^C


In [ ]:
#accuracy

In [21]:

summary_writer = tf.summary.create_file_writer('./tensorboard')
#开启Trace，可以记录图结构和profile信息
tf.summary.trace_on(graph=True, profiler=True)
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)      # 调用模型 y_pred = model(X) 而不是显式写出 y_pred = a * X + b
        loss = tf.reduce_mean(tf.square(y_pred - y))
        with summary_writer.as_default():
            tf.summary.scalar("loss", loss, step=i)

    grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir='./log_dir')    # 保存Trace信息到文件（可选）


In [ ]:
!tensorboard --logdir=./log_dir 

In [25]:
import tensorflow as tf
import numpy as np

X = tf.constant([2013, 2014, 2015, 2016, 2017])
Y = tf.constant([12000, 14000, 15000, 16500, 17500])

# 也可以使用NumPy数组，效果相同
# X = np.array([2013, 2014, 2015, 2016, 2017])
# Y = np.array([12000, 14000, 15000, 16500, 17500])
#dataset = tf.data.Dataset.from_tensor_slices((X, Y))

dataset = tf.data.Dataset.from_tensor_slices((X, Y)).batch(2)

#min-batch SGD
for x, y in dataset:
    print(x, y) 

tf.Tensor([2013 2014], shape=(2,), dtype=int32) tf.Tensor([12000 14000], shape=(2,), dtype=int32)
tf.Tensor([2015 2016], shape=(2,), dtype=int32) tf.Tensor([15000 16500], shape=(2,), dtype=int32)
tf.Tensor([2017], shape=(1,), dtype=int32) tf.Tensor([17500], shape=(1,), dtype=int32)


## @tf.function

In [1]:
import tensorflow as tf
import time
start_time=time.time()


class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )

    def call(self, input):
        output = self.dense(input)
        return output
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])
model = Linear()
def train_on_step(X,y):
    with tf.GradientTape() as tape:
        y_pred = model(X)      # 调用模型 y_pred = model(X) 而不是显式写出 y_pred = a * X + b
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        
for i in range(10000):
    train_on_step(X,y)
    
end_time=time.time()
print(end_time-start_time)

14.47761869430542


In [1]:
import tensorflow as tf
import time
start_time=time.time()


class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )

    def call(self, input):
        output = self.dense(input)
        return output
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
import time
start_time=time.time()

X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])
model = Linear()
@tf.function
def train_on_step(X,y):
    with tf.GradientTape() as tape:
        y_pred = model(X)      # 调用模型 y_pred = model(X) 而不是显式写出 y_pred = a * X + b
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        
for i in range(10000):
    train_on_step(X,y)
    
end_time=time.time()
print(end_time-start_time)

2.7334132194519043
